In [1]:
import pandas as pd
import os
from os import path
import pickle
import json
import numpy as np
from tqdm import tqdm
import math

In [2]:
df = pd.read_csv("../../Dataset/datasetBinary.csv")

In [3]:
train_cols = df.columns.difference(['customer_types', 'conversion', 'treat'])
X = df[train_cols] 
y = df['treat']

In [ ]:
df.head()

In [4]:
targetCol = "treat"

In [5]:
#undersampling imports
from imblearn.under_sampling import RandomUnderSampler
from imblearn.under_sampling import CondensedNearestNeighbour
from imblearn.under_sampling import TomekLinks
from imblearn.under_sampling import EditedNearestNeighbours

In [6]:
#other imports
#data partitioning
from sklearn.model_selection import train_test_split, cross_val_score

#classification
from sklearn.metrics import accuracy_score, f1_score, classification_report
from sklearn.metrics import roc_curve, auc, roc_auc_score
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.decomposition import PCA


In [7]:
X_train, X_test, y_train, y_test = train_test_split(df, y, test_size=0.25, random_state=12, stratify=y)


In [8]:
from collections import Counter

In [9]:
Counter(y)

Counter({1: 42694, 0: 21306})

In [ ]:
from utilsBalancing import makeAndSaveToFileDecisionTreeResults

base dataset classification

In [ ]:
clfs, results = makeAndSaveToFileDecisionTreeResults (X_train, y_train, X_test, y_test, "resampled/based")

unbalanced classes above

In [ ]:
rus = RandomUnderSampler(random_state=42)
X_res, y_res = rus.fit_resample(X_train, y_train)
print('Resampled dataset shape %s' % Counter(y_res))

In [ ]:
clfs, results = makeAndSaveToFileDecisionTreeResults (X_res, y_res, X_test, y_test, "resampled/RandomUnderSampler")

In [10]:
from sklearn.cluster import MiniBatchKMeans
from imblearn.under_sampling import ClusterCentroids

In [11]:
cc = ClusterCentroids(
    estimator=MiniBatchKMeans(
        n_init=1, random_state=42
    ), 
    random_state=42
)
X_res, y_res = cc.fit_resample(X_train, y_train)w
print('Resampled dataset shape %s' % Counter(y_res))

Resampled dataset shape Counter({0: 15979, 1: 15979})


In [12]:
# Concatenate the features and target into a single DataFrame
resampled_df = pd.concat([X_res, y_res], axis=1)

# Save the DataFrame to a CSV file
resampled_df.to_csv('resampled_dataset cluster centroids.csv', index=False)

In [ ]:
clfs, results = makeAndSaveToFileDecisionTreeResults (X_res, y_res, X_test, y_test, "resampled/ClusterCentroids")

In [ ]:
from imblearn.over_sampling import SMOTE


In [ ]:
sm = SMOTE(random_state=42)
X_res, y_res = sm.fit_resample(X_train, y_train)
print('Resampled dataset shape %s' % Counter(y_res))

In [ ]:
clfs, results = makeAndSaveToFileDecisionTreeResults (X_res, y_res, X_test, y_test, "resampled/smote")
